In [51]:
import streamlit as st
from streamlit_autorefresh import st_autorefresh
from datetime import datetime as dt
from datetime import date, timedelta
import pandas as pd
import psycopg2
import altair as alt
import os
import solaredge
from dotenv import load_dotenv
load_dotenv()
alt.renderers.enable('mimetype')

user = os.getenv('HAA_DB_USER')
password = os.getenv('HAA_DB_PASSWORD')
host = os.getenv('HAA_DB_HOST')
se_token = os.getenv('HAA_SOLAREDGE')
se_site_id = os.getenv('HAA_SE_SITE_ID')
s = solaredge.Solaredge(se_token)


def make_connection():
    conn = psycopg2.connect(user=user, password=password, host=host, database='postgres', port=5432)
    return conn

def get_temperature_total():
    conn = make_connection()    
    sql = "select * from temperatuur where tijd > '"+str(dt.today())[0:10]+" 00:00:00';"
    df = pd.read_sql_query(sql, con = conn)[['tijd', 'buiten', 'keuken', 'kamer']].set_index('tijd')
    return df

# temperaturen vandaag
def get_temperature(ruimte):
    conn = make_connection()
    sql = "select * from temperatuur where tijd > '"+str(dt.today())[0:10]+" 00:00:00';"
    df = pd.read_sql_query(sql, con = conn)[['tijd', ruimte]].set_index('tijd')
    df['uur'] = df.index.floor('H')
    df = df.set_index('uur')
    df = df.groupby('uur').mean()
    df['tijd'] = df.index

    return df

buiten = get_temperature('buiten')
kamer = get_temperature('kamer')
keuken = get_temperature('keuken')
alles = get_temperature_total()

alles = buiten.buiten.to_list()+keuken.keuken.to_list()+kamer.kamer.to_list()
min_value = int(min(alles))
max_value = round(max(alles), 0)



/home/kali/git/kitchen/env/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/kali/git/kitchen/env/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/kali/git/kitchen/env/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/kali/git/kitchen/env/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(

In [22]:
buiten['buiten']

uur
2022-04-12 00:00:00    6.174600
2022-04-12 01:00:00    6.176667
2022-04-12 02:00:00    5.968500
2022-04-12 03:00:00    5.562167
2022-04-12 04:00:00    5.291333
2022-04-12 05:00:00    5.100000
2022-04-12 06:00:00    4.478833
2022-04-12 07:00:00    4.824800
2022-04-12 08:00:00    6.572667
2022-04-12 09:00:00    7.546500
Name: buiten, dtype: float64

In [27]:
max(buiten.buiten)
min(buiten.buiten)

1.1666666666666667

In [2]:
## Plots normaliseren

In [25]:

alt.Chart(buiten).mark_line().encode(
    x=alt.X('tijd'),
    y=alt.Y('buiten', scale=alt.Scale(domain=[min_value, max_value]))
)

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [15]:
buiten.buiten.to_list()

[6.1746,
 6.176666666666667,
 5.9685,
 5.562166666666667,
 5.291333333333333,
 5.1,
 4.478833333333333,
 4.824800000000001,
 6.572666666666667,
 7.5465]

alt.Chart(df).l

In [19]:
alt.Chart(kamer).mark_line().encode(
    x=alt.X('tijd'),
    y=alt.Y('kamer', scale=alt.Scale(domain=[min_value, max_value]))
)

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


## solaredge

In [21]:
def get_energy(begin, end):
    res = s.get_energy_details(se_site_id, begin, end, time_unit='HOUR')
    
    return res

In [39]:
yesterday = date.today() - timedelta(days=1)
se_energy = get_energy(str(yesterday)+" 00:00:00", str(yesterday)+" 23:59:59")

In [42]:
se_date = []
se_value = []

for i in se_energy['energyDetails']['meters'][0]['values']:
    se_date.append(i['date'])
    if len(i) == 1:
        se_value.append(None)
    else:
        se_value.append(round(i['value'], 1))

se_df = pd.DataFrame({'se_date': se_date, 'se_value': se_value})

se_df['day'] = se_df.se_date.str[0:10]
se_df['hour'] = se_df.se_date.str[10:13]



In [71]:
alt.Chart(se_df).mark_line().encode(
    x='hour',
    y='se_value',
    #color = 'day',
    color=alt.Color('day',scale=alt.Scale(range= ['#1f77b4', 'grey']))
)
#st.altair_chart(line_se)

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [58]:
se_df

,se_date,se_value,day
0,2022-04-11 00:00:00,0.0,2022-04-11
1,2022-04-11 01:00:00,0.0,2022-04-11
2,2022-04-11 02:00:00,0.0,2022-04-11
3,2022-04-11 03:00:00,0.0,2022-04-11
4,2022-04-11 04:00:00,0.0,2022-04-11
5,2022-04-11 05:00:00,0.0,2022-04-11
6,2022-04-11 06:00:00,0.0,2022-04-11
7,2022-04-11 07:00:00,27.6,2022-04-11
8,2022-04-11 08:00:00,125.1,2022-04-11
9,2022-04-11 09:00:00,413.2,2022-04-11


In [62]:
se_df.se_date.str[10:13]

0      00
1      01
2      02
3      03
4      04
5      05
6      06
7      07
8      08
9      09
10     10
11     11
12     12
13     13
14     14
15     15
16     16
17     17
18     18
19     19
20     20
21     21
22     22
23     23
24     00
25     01
26     02
27     03
28     04
29     05
30     06
31     07
Name: se_date, dtype: object

In [ ]:
alt.Chart(kamer).mark_line().encode(
    x=alt.X('tijd'),
    y=alt.Y('kamer', scale=alt.Scale(domain=[min_value, max_value]))
)